# Tutorial on JIT versus Scipy execution within Parcels

This very brief tutorial is meant to highlight the potentially very big difference between the computational time required to run Parcels in JIT (Just-In-Time compliation) versus in Scipy mode.

In the code snippet below, we use `AdvectionRK4` to advect 100 particles in the peninsula `FieldSet`. We first do it in JIT mode (by setting `ptype=JITParticle` in the declaration of `pset`) and then we also do it in Scipy mode (by setting `ptype=ScipyParticle` in the declaration of `pset`).

In both cases, we advect the particles for 1 hour, with a timestep of 30 seconds. 

In [2]:
from parcels import FieldSet, ParticleSet, JITParticle, ScipyParticle
from parcels import AdvectionRK4
from datetime import timedelta as delta
import time

fieldset = FieldSet.from_nemo('Peninsula_data/peninsula', allow_time_extrapolation=True)

ptype = {'scipy': ScipyParticle, 'jit': JITParticle}

for p in ['jit', 'scipy']:
    pset = ParticleSet.from_line(fieldset=fieldset, pclass=ptype[p], size=100, start=(0.1, 0.1), finish=(0.1, 0.4))

    t0 =time.time()
    pset.execute(AdvectionRK4, runtime=delta(hours=1), dt=delta(seconds=30))
    print p, 'execution took', format(time.time()-t0, '.2f'), 'seconds'

INFO: Compiled JITParticleAdvectionRK4 ==> /var/folders/r2/8593q8z93kd7t4j9kbb_f7p00000gn/T/parcels-501/4de448cefdb0e5792c912beafdd4470e.so


jit execution took 0.17 seconds
scipy execution took 17.46 seconds


As you can see above, **Scipy mode took more than 100 times as long** (0.17 seconds versus 17.46 seconds) as the JIT mode!!

This is just an illustrative example, depending on the number of calls to `AdvectionRK4`, the size of the `FieldSet`, the size of the `pset`, the ratio between `dt` and `interval` in the `.execute` etc, the difference between JIT and Scipy can vary significantly. However, JIT will almost always be faster!

So why does Parcels support both JIT and Scipy mode then? Because Scipy is easier to debug when writing custom kernels, so can provide faster development of new features.

*As an aside, you may wonder why we use the `time.time` module, and not the timeit module, to time the runs above. That's because it affects the AST of the kernels, causing errors in JIT mode.*